![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
# Chapter 4: Identifying the blades

## Dependencies

In [ ]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If Numba is not installed
%pip install numba
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

## Imports

In [ ]:
from bladesight import Datasets
from bladesight.btt.aoa import transform_ToAs_to_AoAs
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numba import njit
import pandas as pd
import numpy as np
from typing import Tuple, List

## AoA Histogram

In [ ]:
dataset = Datasets['data/intro_to_btt/intro_to_btt_ch03']
df_opr_zero_crossings = \
    dataset[f"table/du_toit_2017_test_1_opr_zero_crossings"]
df_prox_1_toas = dataset[f"table/du_toit_2017_test_1_prox_1_toas"]

df_prox_1 = transform_ToAs_to_AoAs(
    df_opr_zero_crossings,
    df_prox_1_toas
)

blade_arrival_count, histogram_bins = np.histogram(
    df_prox_1["AoA"],
    bins=np.linspace(0, 2*np.pi, 50)
)

In [ ]:
bin_count, bin_edges = np.histogram(df_prox_1["AoA"], bins=np.linspace(0, 2*np.pi, 50))
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=(histogram_bins[:-1] + histogram_bins[1:])/2 * 180/np.pi,
        y=blade_arrival_count,
        name="AoA histogram"
    )
)
# Set x range to between 0 and 2pi
fig.update_xaxes(range=[0, 360])

fig.update_layout(
    title="AoA histogram",
    xaxis_title="AoA [deg]",
    yaxis_title="Count"
)


In [ ]:
def calculate_Q(
    arr_aoas : np.ndarray,
    d_theta : float,
    N : int
) -> Tuple[float, np.ndarray]:
    bin_edges = np.linspace(0 + d_theta, 2*np.pi + d_theta, N + 1)
    Q = 0
    for b in range(N):
        left_edge = bin_edges[b]
        right_edge = bin_edges[b + 1]
        bin_mask = (arr_aoas > left_edge) & (arr_aoas <= right_edge)

        bin_centre = (left_edge + right_edge)/2
        Q += np.sum(
            (
                arr_aoas[bin_mask] 
                - bin_centre
            )**2 
        )
    if np.sum(arr_aoas < bin_edges[0]) > 0:
        return np.nan, bin_edges
    if np.sum(arr_aoas > bin_edges[-1]) > 0:
        return np.nan, bin_edges
    return Q, bin_edges


## Implementation example

In [ ]:
B = 5
d_thetas = np.linspace(-np.pi/B, np.pi/B, 200) 
arr_aoas = df_prox_1["AoA"].to_numpy()
Qs = [] 
optimal_Q, optimal_bin_edges, optimal_d_theta = np.inf, None, None
for d_theta in d_thetas:
    Q, bin_edges = calculate_Q(arr_aoas, d_theta, B)
    if Q < optimal_Q:
        optimal_Q = Q*1
        optimal_bin_edges = bin_edges
        optimal_d_theta = d_theta*1
    Qs.append(Q)


In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=d_thetas * 180/np.pi,
        y=Qs,
        name="Q values"
    )
)
fig.add_trace(
    go.Scatter(
        x=[optimal_d_theta * 180/np.pi],
        y=[optimal_Q],
        name="Optimal d_theta value= {:.2f}°".format(optimal_d_theta*180/np.pi),
        mode="markers",
        marker={
            "size": 10
        }
    )
)

fig.update_layout(
    title="Q values for different d_theta values",
    xaxis_title="d_theta [deg]",
    yaxis_title="Q"
)
fig.show()


## Grouping the blades


In [ ]:
blade_dfs = []
for b in range(B):
    ix_bin = (
        (df_prox_1["AoA"] > optimal_bin_edges[b])
        & (df_prox_1["AoA"] <= optimal_bin_edges[b + 1])
    )
    blade_dfs.append(
        df_prox_1.loc[ix_bin]
    )


In [ ]:
for b in range(B):
    print(f"Blade {b} mean: {blade_dfs[b]['AoA'].mean()}, std: {blade_dfs[b]['AoA'].std()}")

## Wrapping blades

In [ ]:
df_prox_1_shifted = df_prox_1.copy(deep=True)
df_prox_1_shifted['AoA'] = df_prox_1_shifted['AoA'] - 0.280844143512115
df_prox_1_shifted['AoA'] = df_prox_1_shifted['AoA'] % (2*np.pi)

B = 5
d_thetas = np.linspace(-np.pi/B, np.pi/B, 200)
arr_aoas = df_prox_1_shifted["AoA"].to_numpy()
Qs = []
optimal_Q, optimal_bin_edges, optimal_d_theta = np.inf, None, None
for d_theta in d_thetas:
    Q, bin_edges = calculate_Q(arr_aoas, d_theta, B)
    if Q < optimal_Q:
        optimal_Q = Q*1
        optimal_bin_edges = bin_edges
        optimal_d_theta = d_theta*1
    Qs.append(Q)


In [ ]:
print(optimal_Q)
print(optimal_bin_edges)


## Coding exercises

In [ ]:
# Your turn 👇
def calculate_Q(
    arr_aoas : np.ndarray,
    d_theta : float,
    N : int
) -> Tuple[float, np.ndarray]:
    # Please complete me
    ...

In [ ]:
# Your turn 👇
def transform_prox_AoAs_to_blade_AoAs(
    df_prox : pd.DataFrame,
    B : int,
) -> List[pd.DataFrame]:
    # Please complete me
    ...